# Notes/Ideas



In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
from wordcloud import WordCloud
import pandas as pd
import numpy as np

import json

import requests

from bs4 import BeautifulSoup
import re

import time

import nltk
import string

plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('fivethirtyeight')
plt.style.use('ggplot')

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Faculty Profiles data

In [3]:
df_faculty = pd.read_csv('../scraped_data/faculty_profiles.csv', sep='~')

In [4]:
df_faculty.head()

,faculty_name,faculty_profile_url,l_expertise,department,title_name,faculty_site_url,lab_url,faculty_email,description,description_links,...,title_of_news_4,link_to_news_4,description_teaser_4,article_date_5,title_of_news_5,link_to_news_5,description_teaser_5,first_name,last_name,full_name
0,David A. Aaker,/faculty/david-aaker,"business,marketing,branding",Haas School of Business,Professor of Marketing and Public Policy,http://www.haas.berkeley.edu/faculty/aaker.html,http://groups.haas.berkeley.edu/marketing/,aaker@haas.berkeley.edu,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,david,aaker,david aaker
1,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,Big NSF grant funds research into training ro...,/news/big-nsf-grant-funds-research-training-ro...,"What if robots and humans, working together,...","August 23, 2011",UC Berkeley robotics expert named among world...,/news/uc-berkeley-robotics-expert-named-among-...,"Pieter Abbeel, a UC Berkeley, professor know...",pieter,abbeel,pieter abbeel
2,Elizabeth Abel,/faculty/elizabeth-abel,"feminist theory,psychoanalysis,virginia woolf,...",Department of English,Professor of English,http://english.berkeley.edu/profiles/5,NaN,eabel@uclink.berkeley.edu,Elizabeth Abel's general research interest is...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,elizabeth,abel,elizabeth abel
3,Dor Abrahamson,/faculty/dor-abrahamson,"mathematical cognition,design-based research,m...",Graduate School of Education,Associate Professor of Cognition and Development,http://gse.berkeley.edu/people/dor-abrahamson,http://edrl.berkeley.edu/,dor@berkeley.edu,Dor Abrahamson studies the process of mathema...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dor,abrahamson,dor abrahamson
4,Norman Abrahamson,/faculty/norman-abrahamson,"civil and environmental engineering,earthquake...",Department of Civil and Environmental Engineering,Adjunct Professor of Civil and Environmental E...,http://www.ce.berkeley.edu/faculty/faculty.php...,NaN,naa3@earthlink.net,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,norman,abrahamson,norman abrahamson


In [5]:
df_faculty.columns.values

array(['faculty_name', 'faculty_profile_url', 'l_expertise', 'department',
       'title_name', 'faculty_site_url', 'lab_url', 'faculty_email',
       'description', 'description_links', 'article_date_1',
       'title_of_news_1', 'link_to_news_1', 'description_teaser_1',
       'article_date_2', 'title_of_news_2', 'link_to_news_2',
       'description_teaser_2', 'article_date_3', 'title_of_news_3',
       'link_to_news_3', 'description_teaser_3', 'article_date_4',
       'title_of_news_4', 'link_to_news_4', 'description_teaser_4',
       'article_date_5', 'title_of_news_5', 'link_to_news_5',
       'description_teaser_5', 'first_name', 'last_name', 'full_name'], dtype=object)

In [6]:
def split_first_last_name(s):
    """
    Find highest two 'words', those are the first and last name by order
    """
    l = s.lower().split(' ')
    lengths = [len(w) for w in l]
    max_index = max(range(len(l)), key=lambda i:lengths[i])
    max_word = l[max_index]
    lengths[max_index] = -1
    
    max_index2 = max(range(len(l)), key=lambda i:lengths[i])
    max_word2 = l[max_index2]
    
    if max_index < max_index2:
        return max_word, max_word2
    else:
        return max_word2, max_word
    
    

first_last_names = df_faculty['faculty_name'].apply(split_first_last_name)
first_names = [t[0] for t in first_last_names]
last_names = [t[1] for t in first_last_names]
df_faculty['first_name'] = first_names
df_faculty['last_name'] = last_names
df_faculty['full_name'] = df_faculty['first_name'] + ' ' + df_faculty['last_name']

In [7]:
bool_filter = ~df_faculty['l_expertise'].isnull()
df_faculty = df_faculty[bool_filter]
df_faculty['l_expertise'] = df_faculty['l_expertise'].apply(lambda s:s.lower())

In [8]:
len(set(df_faculty['department']))

72

In [9]:
set(df_faculty['department'])

{'Boalt Hall School of Law',
 'College of Engineering',
 'College of Letters and Science',
 'College of Natural Resources',
 'Department of African American Studies',
 'Department of Agricultural & Resource Economics',
 'Department of American Studies',
 'Department of Anthropology',
 'Department of Architecture',
 'Department of Art Practice',
 'Department of Astronomy',
 'Department of Bioengineering',
 'Department of Chemical Engineering',
 'Department of Chemistry',
 'Department of City & Regional Planning',
 'Department of Civil and Environmental Engineering',
 'Department of Classics',
 'Department of Comparative Literature',
 'Department of Demography',
 'Department of Earth and Planetary Science',
 'Department of East Asian Languages and Cultures',
 'Department of Economics',
 'Department of English',
 'Department of Environmental Science, Policy & Management',
 'Department of Ethnic Studies',
 'Department of Film and Media',
 'Department of French',
 "Department of Gender and 

In [207]:
finished_departments = set([
 'College of Engineering',
 'College of Letters and Science',
 'College of Natural Resources',
 'Department of Agricultural & Resource Economics',
 'Department of Astronomy',
 'Department of Bioengineering',
 'Department of Chemical Engineering',
 'Department of Chemistry',
 'Department of City & Regional Planning',
 'Department of Civil and Environmental Engineering',
 'Department of Economics',
 'Department of Environmental Science, Policy & Management',
 'Department of History',
 'Department of Industrial Engineering & Operations Research',
 'Department of Integrative Biology',
 'Department of Materials Science and Engineering',
 'Department of Mathematics',
 'Department of Molecular & Cell Biology',
 'Department of Nuclear Engineering',
 'Department of Nutritional Sciences & Toxicology',
 'Department of Physics',
 'Department of Plant and Microbial Biology',
 'Department of Statistics',
 'Division of Computer Science/EECS',
 'Division of Electribcal Engineering/EECS',
 'Environmental Science, Policy & Management',
 'Department of Earth and Planetary Science',
 'Division of Electrical Engineering/EECS',
 'Haas School of Business',
 'Helen Wills Neuroscience Institute',
 'Energy & Resources Group',
 'Department of Psychology',
 'Department of Political Science',
 'Department of Sociology',
 'Department of Anthropology',
 'Goldman School of Public Policy',
 'Department of Linguistics',
 'School of Public Health',
 'School of Social Welfare',
 'Landscape Architecture & Environmental Planning',
 'School of Information',
 'Department of Mechanical Engineering',
 'Department of African American Studies',
 'Department of Architecture',
 'Department of Comparative Literature',
 'Department of English',
 'Department of Geography',
 'Department of Rhetoric',
 'Department of Philosophy',
])

### Todo list

In [86]:
set(df_faculty['department']) - finished_departments

{'Boalt Hall School of Law',
 'Department of American Studies',
 'Department of Art Practice',
 'Department of Classics',
 'Department of Demography',
 'Department of East Asian Languages and Cultures',
 'Department of Ethnic Studies',
 'Department of Film and Media',
 'Department of French',
 "Department of Gender and Women's Studies",
 'Department of German',
 'Department of History of Art',
 'Department of Italian Studies',
 'Department of Music',
 'Department of Near Eastern Studies',
 'Department of Scandinavian',
 'Department of Slavic Languages and Literatures',
 'Department of South and Southeast Asian Studies',
 'Department of Spanish and Portuguese',
 'Graduate School of Education',
 'Interdisciplinary Studies (ISF)',
 'School of Journalism',
 'School of Optometry',
 'Theatre, Dance, and Performance Studies'}

### Departments/Colleges/Institutes that are covered by smaller departments/mix of other departments

'Helen Wills Neuroscience Institute' is made up of mcb and other related departments

'Energy & Resources Group' espm, mcb, etc

### A lot of the faculty profiles don't work or don't point to the right source

In [14]:
df_faculty['faculty_site_url'].values[:30]

array(['http://www.haas.berkeley.edu/faculty/aaker.html',
       'http://www.cs.berkeley.edu/~pabbeel',
       'http://english.berkeley.edu/profiles/5',
       'http://gse.berkeley.edu/people/dor-abrahamson',
       'http://www.ce.berkeley.edu/faculty/faculty.php?id=780',
       'http://history.berkeley.edu/people/richard-m-abrams',
       'http://sph.berkeley.edu/barbara-abrams-drph-rd',
       'http://www.law.berkeley.edu/faculty/profiles/facultyProfile.php?facID=1105',
       'http://ced.berkeley.edu/ced/faculty-staff/charisma-acey', nan,
       'https://gspp.berkeley.edu/directories/faculty/dan-acland',
       'http://cci.lbl.gov/paul',
       'http://optometry.berkeley.edu/faculty/anthony-j-adams',
       'http://math.berkeley.edu/people/faculty/john-w-addison',
       'http://www.ieor.berkeley.edu/People/Faculty/adler.htm',
       'http://physics.berkeley.edu/people/faculty/mina-aganagic',
       'http://anthropology.berkeley.edu/users/sabrina-c-agarwal',
       'http://polisci.b

In [10]:
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}

def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    elif element == '\n':
        return False
    return True

def scrape_urls(url, fn, base_url=''):
    """
    Get the urls of faculty pages from a url that has a listing of all faculty urls
    """
    if type(url) == type([]):
        url_l = url
        total_urls = {}
        for url in url_l:
            try:
                soup = BeautifulSoup(requests.get(url, headers=headers).content)
                urls = fn(soup)
                for faculty in urls:
                    total_urls[faculty] = base_url + urls[faculty]
            except:
                print(url)
        return total_urls

    soup = BeautifulSoup(requests.get(url, headers=headers).content)
    urls = fn(soup)
    for faculty in urls:
        urls[faculty] = base_url + urls[faculty]
    return urls

def feed_me_soup(soup, use_div, class_name=None, id_name=None):
    selector = 'div' if use_div else 'section'
    try:
        if id_name is not None:
            dump = list(filter(visible, soup.find_all(selector, {'id':id_name})[0].find_all(text=True)))
            return ' '.join([t for t in [t.replace(u'\xa0', u'').strip() for t in dump] if t != ''])
        dump = list(filter(visible, soup.find_all(selector, {'class':class_name})[0].find_all(text=True)))
        return ' '.join([t for t in [t.replace(u'\xa0', u'').strip() for t in dump] if t != ''])
    except:
        dump = list(filter(visible, soup.find_all(text=True)))
        return ' '.join([t for t in [t.replace(u'\xa0', u'').strip() for t in dump] if t != ''])


def scrape_faculty_page_general(url):
    try:
        soup = BeautifulSoup(requests.get(url, headers=headers).content)
        dump = list(filter(visible, soup.find_all(text=True)))
        return ' '.join([t for t in [t.replace(u'\xa0', u'').strip() for t in dump] if t != ''])
    except:
        return None


def scrape_faculty_page(url, department_name):
    """
    Gets the content of a faculty webpage
    """
    soup = BeautifulSoup(requests.get(url, headers=headers).content)
    use_div = True
    if 'section' in departments_d[department_name]:
        use_div = False
        
    if 'use_content_id' in departments_d[department_name]:
        dump = feed_me_soup(soup, use_div, id_name=departments_d[department_name]['content_fn'])
    else:
        dump = feed_me_soup(soup, use_div, class_name=departments_d[department_name]['content_fn'])
    return dump

In [96]:
def get_philosophy_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'PersonListing'})
    for div in divs:
        a = div.find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_rhetoric_urls(soup):
    urls = {}
    ahrefs = soup.find_all('div', {'class':'body-main'})[0].find_all('a')
    for a in ahrefs:
        url = a['href']
        name = a.find_all('h3')[0].text
        urls[name] = url
    return urls

def get_geography_urls(soup):
    urls = {}
    articles = soup.find_all('div', {'id':'main'})[0].find_all('article')
    for article in articles:
        a = article.find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_english_urls(soup):
    urls = {}
    trs = soup.find_all('div', {'id':'active_faculty'})[0].find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_complit_urls(soup):
    urls = {}
    sections = soup.find_all('section', {'class':'content-container'})
    for section in sections:
        articles = section.find_all('article')
        for article in articles:
            a = article.find_all('h1')[0].find_all('a')[0]
            urls[a.text.strip()] = a['href']
    return urls

def get_afraamer_urls(soup):
    urls = {}
    tds = soup.find_all('div', {'class':'view-content'})[0].find_all('td')
    for td in tds:
        a = td.find_all('span')[0].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_me_urls(soup):
    urls = {}
    lis = soup.find_all('div', {'class':'view-content'})[0].find_all('ul')[0].find_all('li', recursive=False)
    for li in lis:
        a = li.find_all('span')[0].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_school_of_info_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_social_welfare_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'block-system-main'})[0].find_all('div', {'class':'view-content'})[0]\
        .find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_public_health_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'content'})[0].find_all('div', {'class':'view-content'})[0]\
        .find_all('div', recursive=False)
    for div in divs:
        block = div.find_all('div', recursive=False)
        name = block[1].find_all('h3')[0].text
        name = name.replace(',', '').replace('MD', '').replace('DrPH', '')\
                .replace('PhD', '').replace('MA', '')\
                .replace('MS', '').replace('MPH', '').replace('MRP', '').replace('RD', '').strip()
        try:
            url = block[3].find_all('a')[0]['href']
            urls[name] = url
        except:
            pass
    return urls

def get_linguistics_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'main-inner'})[0].find_all('div', {'class':'panel-panel-inner'})[0]\
        .find_all('div', recursive=False)
    for div in divs:
        try:
            a = div.find_all('h2')[0].find_all('a')[0]
            url = a['href']
            if url.startswith('/'):
                url = 'http://lx.berkeley.edu{}'.format(url)
            urls[a.text] = url
        except:
            pass
    return urls

def get_anthro_urls(soup):
    urls = {}
    trs = soup.find_all('div', {'class':'view-content'})[1].find_all('table')[0].find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[0].find_all('div')[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_sociology_urls(soup):
    urls = {}
    trs = soup.find_all('div', {'class':'view-content'})[0].find_all('table')[0].find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[0].find_all('a')[0]
        urls[a.text] = a['href']

    return urls

def get_poli_sci_urls(soup):
    urls = {}
    trs = soup.find_all('div', {'class':'view-content'})[0].find_all('table')[0].find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[0].find_all('a')[0]
        urls[a.text] = a['href']

    return urls

def get_psych_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
    for div in divs:
        block = div.find_all('div', {'class':'panel-panel panel-col'})[0]
        a = block.find_all('a')[0]
        spans = block.find_all('span')
        urls['{} {}'.format(spans[0].text.strip(), spans[1].text.strip())] = a['href']
    return urls

def get_gssp_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'content'})[0].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('h3')[0].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_haas_urls(soup):
    urls = {}
    blocks = soup.find_all('section', {'id':'center-col'})[0].find_all('div', {'class':'faculty-block'})
    for div in blocks:
        divs = div.find_all('div', recursive=False)
        for div in divs:
            a = div.find_all('p')[0].find_all('a')[0]
            last, first = split_first_last_name(a.text.replace('\n', '').replace('\t', '').replace(',', '').strip())
            urls['{} {}'.format(first, last)] = a['href']
    urls['john phillips'] = urls['(jack) phillips']
    del urls['(jack) phillips']
    return urls

def get_nuc_eng_urls(soup):
    urls = {}
    for a in soup.find_all('div', {'class':'view-content'})[0].find_all('a'):
        urls[a.text] = a['href']
    del urls['']
    return urls

def get_mse_urls(soup):
    urls = {}
    tables = soup.find_all('table', {'class':'facultytable'})
    for table in tables:
        tds = table.find_all('td')
        for td in tds:
            try:
                aas = td.find_all('a')
                if len(aas) > 1:
                    a = aas[1]
                else:
                    a = aas[0]
                urls[a.text.strip().replace('\xa0', ' ')] = a['href'].strip()
            except Exception as e:
                pass
    return urls

def get_eps_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        ps = div.find_all('p')
        for p in ps:
            a = p.find_all('a')[0]
            urls[a.text] = a['href'][5:]
    return urls

def get_city_regional_planning_urls(soup):
    urls = {}
    lis = soup.find_all('div', {'id':'faculty'})[0].find_all('li')
    for li in lis:
        urls[li.find_all('h2')[0].text.replace('  ', ' ')] = li.find_all('a')[0]['href']
    return urls

def get_astro_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'row faculty-list researcher-list clearfix'})[0].find_all('div')
    for div in divs:
        a = div.find_all('a')[1]
        urls[a.text] = a['href']
    return urls

def get_plant_micro_bio_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_nutri_sci_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_physics_urls(soup):
    urls = {}
    uls = soup.find_all('div', {'class':'view-content'})[1].find_all('ul')
    for ul in uls:
        lis = ul.find_all('li')
        for li in lis:
            a = li.find_all('a')[1]
            urls[a.text] = a['href']
    return urls

def get_ib_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('a')[0]
        urls[a.text.replace('  ', ' ')] = a['href']
    return urls

def get_env_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text.replace('  ', ' ')] = a['href']
    return urls

def get_are_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'content'})[0].find_all('div', recursive=False)[1:]
    for div in divs:
        subdivs = div.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
        for subdiv in subdivs:
            a = subdiv.find_all('div', recursive=False)[1].find_all('a')[0]
            urls[a.text] = a['href']
    return urls

def get_econ_urls(soup):
    urls = {}
    trs = soup.find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[1].find_all('a')[0]
        last, first = a.text.strip().split(', ')
        urls[first + ' ' + last] = a['href']
    return urls

def get_chem_urls(soup):
    ahrefs = soup.find_all('div', {'class':'content'})[0]\
            .find_all('div', recursive='False')[0].find_all('div', recursive='False')[0]\
                .find_all('div', recursive='False')[0].find_all('a')
    urls = {}
    for a in ahrefs:
        try:
            if a['href'].startswith('http://'):
                last, first = a.text.split(', ')
                urls[first + ' ' + last] = a['href']
        except:
            pass
    return urls

def get_ieor_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'faculty-group clearfix'})
    for div in divs:
        lis = div.find_all('ol')[0].find_all('li', recursive=False)
        for li in lis:
            a = li.find_all('div')[2].find_all('a')[0]
            urls[a.text] = a['href']
    urls['J George Shanthikumar'] = urls[' \tJ George Shanthikumar ']
    del urls[' \tJ George Shanthikumar ']
    return urls

def get_civil_env_e_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div')
    for div in divs:
        try:
            a = div.find_all('div')[1].find_all('a')[0]
            urls[a.text.replace('  ', ' ')] = a['href']
        except:
            pass
    return urls

def get_bioe_urls(soup):
    urls = {}
    ahrefs = soup.find_all('div', {'class':'entry-content'})[0].find_all('a')
    for a in ahrefs:
        urls[a.find_all('strong')[0].text] = a['href']
    return urls

def get_mcb_urls(soup):
    urls = {}
    ahrefs = soup.find_all('div', {'id':'block-system-main'})[0].find_all('a')
    for a in ahrefs:
        try:
            url = a['href']
            if url.startswith('/'):
                urls[a.text] = url
        except:
            pass
    del urls['Faculty by Division']
    del urls['More Information >']
    return urls

def get_statistics_urls(soup):
    urls = {}
    tds = soup.find_all('tbody')[0].find_all('td')
    for td in tds:
        try:
            a = td.find_all('div')[2].find_all('a')
            urls[a[0].text] = a[0]['href']
        except:
            pass
    return urls

def get_math_urls(soup):
    urls = {}
    tds = soup.find_all('tbody')[0].find_all('td')
    for td in tds:
        try:
            a = td.find_all('div')[1].find_all('a')
            urls[a[0].text] = a[0]['href']
        except:
            pass
    return urls

def get_eecs_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'media'})
    for div in divs:
        a = div.find_all('a')
        urls[a[1].text] = a[0]['href']
    return urls

def get_history_urls(soup):
    urls = {}
    trs = soup.find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('a')[0]
        urls[a.text] = a['href']
    return urls

SyntaxError: invalid syntax (<ipython-input-96-cd7c758eed7b>, line 50)

In [ ]:
departments_d = {
    'history': {
        'url':'http://history.berkeley.edu/faculty',
        'base':'http://history.berkeley.edu',
        'url_fn':get_history_urls,
        'content_fn':'region region-content'
    },
    'eecs': {
        'url':'https://www2.eecs.berkeley.edu/Faculty/Lists/list.html?_ga=1.58368611.858746805.1441508156',
        'base':'https://www2.eecs.berkeley.edu',
        'url_fn':get_eecs_urls,
        'content_fn':'content'
    },
    'math': {
        'url':'https://math.berkeley.edu/people/faculty',
        'base':'https://math.berkeley.edu',
        'url_fn':get_math_urls,
        'content_fn':'content'
    },
    'stats': {
        'url':'http://statistics.berkeley.edu/people/faculty',
        'base':'http://statistics.berkeley.edu',
        'url_fn':get_statistics_urls,
        'content_fn':'content-wrapper',
        'use_content_id':True
    },
    'mcb': {
        'url':'https://mcb.berkeley.edu/faculty/all',
        'base':'https://mcb.berkeley.edu',
        'url_fn':get_mcb_urls,
        'content_fn':'content',
        'use_content_id':True
    },
    'bioe': {
        'url':'http://bioeng.berkeley.edu/people',
        'url_fn':get_bioe_urls,
        'content_fn':'entry-content'
    },
    'civil_env_e': {
        'url':'http://www.ce.berkeley.edu/people/faculty',
        'base':'http://www.ce.berkeley.edu',
        'url_fn':get_civil_env_e_urls,
        'content_fn':'view-content'
    },
    'ieor': {
        'url':'http://ieor.berkeley.edu/people/faculty',
        'base':'http://ieor.berkeley.edu',
        'url_fn':get_ieor_urls,
        'content_fn':'content',
        'use_content_id':True
    },
    'chem': {
        'url':'http://chemistry.berkeley.edu/faculty/chem',
        'url_fn':get_chem_urls,
        'content_fn':'content'
    },
    'cheme': {
        'url':'http://chemistry.berkeley.edu/faculty/cbe',
        'url_fn':get_chem_urls,
        'content_fn':'content'
    },
    'econ': {
        'url': 'https://www.econ.berkeley.edu/faculty',
        'url_fn':get_econ_urls,
        'content_fn':'view-content'
    },
    'are': {
        'url':'https://are.berkeley.edu/people/faculty',
        'base':'https://are.berkeley.edu',
        'url_fn':get_are_urls,
        'content_fn':'view-content'
    },
    'env': {
        'url':'https://ourenvironment.berkeley.edu/people/faculty-specialists',
        'base':'https://ourenvironment.berkeley.edu',
        'url_fn':get_env_urls,
        'content_fn':'view-content'
    },
    'ib': {
        'url':'http://ib.berkeley.edu/research/interests',
        'base':'https://ib.berkeley.edu',
        'url_fn':get_ib_urls,
        'content_fn':'node-content'
    },
    'astro_physics': {
        'url':'http://physics.berkeley.edu/research/astrophysics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'molecular_physics': {
        'url':'http://physics.berkeley.edu/research/atomic-molecular-and-optical-physics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'biophysics': {
        'url':'http://physics.berkeley.edu/research/biophysics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'condensed_matter_physics': {
        'url':'http://physics.berkeley.edu/research/condensed-matter-physics-and-materials-science',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'nuclear_physics': {
        'url':'http://physics.berkeley.edu/research-faculty/nuclear-physics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'particle_physics': {
        'url':'http://physics.berkeley.edu/research/particle-physics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'plasma_physics': {
        'url':'http://physics.berkeley.edu/research/plasma-and-nonlinear-dynamics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'nutri_sci': {
        'url':'http://nst.berkeley.edu/people/faculty',
        'base':'http://nst.berkeley.edu',
        'url_fn':get_nutri_sci_urls,
        'content_fn':'view-content'
    },
    'plant_micro_bio': {
        'url':'http://plantandmicrobiology.berkeley.edu/faculty/directory?_ga=1.25912820.858746805.1441508156',
        'base':'http://plantandmicrobiology.berkeley.edu',
        'url_fn':get_plant_micro_bio_urls,
        'content_fn':'entity entity-profile2 profile2-main clearfix'
    },
    'astro': {
        'url':'http://astro.berkeley.edu/people/researchers',
        'base':'http://astro.berkeley.edu',
        'url_fn':get_astro_urls,
        'content_fn':'detail-body'
    },
#     'city_regional_planning': {
#         'url':'https://ced.berkeley.edu/academics/city-regional-planning/people/faculty/',
#         'url_fn':get_city_regional_planning_urls,
#         'content_fn':'profile-content'
#     },
    'eps': {
        'url':'http://eps.berkeley.edu/research',
        'base':'http://eps.berkeley.edu/people',
        'url_fn':get_eps_urls,
        'content_fn':'ds-2col-stacked user-profile view-mode-full clearfix'
    },
    'mse': {
        'url':'https://www.mse.berkeley.edu/people/faculty',
        'url_fn':get_mse_urls,
        'content_fn':'entry-content'
    },
    'nuc_eng': {
        'url':'http://www.nuc.berkeley.edu/people/faculty',
        'base':'http://www.nuc.berkeley.edu',
        'url_fn':get_nuc_eng_urls,
        'content_fn':'content'
    },
    'haas': {
        'url':'http://facultybio.haas.berkeley.edu/faculty-photo/',
        'url_fn':get_haas_urls,
        'content_fn':'center-col',
        'use_content_id':True,
        'section':True
    },
    'gssp': {
        'url':'https://gspp.berkeley.edu/directories/faculty',
        'url_fn':get_gssp_urls,
        'content_fn':'content',
        'use_content_id':True,
    },
    'psych': {
        'url':'http://psychology.berkeley.edu/people/faculty',
        'base':'http://psychology.berkeley.edu/',
        'url_fn':get_psych_urls,
        'content_fn':'card-content',
    },
    'poli_sci': {
        'url':['http://polisci.berkeley.edu/people/faculty', 'http://polisci.berkeley.edu/people/faculty?page=1'],
        'base':'http://polisci.berkeley.edu',
        'url_fn':get_poli_sci_urls,
        'content_fn':'profile',
    },
    'sociology': {
        'url':'http://sociology.berkeley.edu/regular-faculty',
        'base':'http://sociology.berkeley.edu',
        'url_fn':get_sociology_urls,
        'content_fn':'profile',
    },
    'anthro': {
        'url':'http://anthropology.berkeley.edu/faculty',
        'base':'http://anthropology.berkeley.edu',
        'url_fn':get_anthro_urls,
        'content_fn':'user-profile-view',
        'use_content_id':True
    },
    'linguistics': {
        'url':'http://lx.berkeley.edu/people/faculty',
        'url_fn':get_linguistics_urls,
        'content_fn':'main-inner',
        'use_content_id':True
    },
    'public_health': {
        'url':['http://sph.berkeley.edu/faculty-profiles']+\
            ['http://sph.berkeley.edu/faculty-profiles?title=&page={}'.format(i) for i in range(1, 6)],
        'base':'http://sph.berkeley.edu',
        'url_fn':get_public_health_urls,
        'content_fn':'content',
        'use_content_id':True
    },
    'social_welfare': {
        'url':['http://socialwelfare.berkeley.edu/faculty-profiles',
            'http://socialwelfare.berkeley.edu/academic-researchers'],
        'base':'http://socialwelfare.berkeley.edu',
        'url_fn':get_social_welfare_urls,
        'content_fn':'block-system-main',
        'use_content_id':True
    },
    'ced': {
        'url':'https://ced.berkeley.edu/ced/faculty-staff/',
        'url_fn':get_city_regional_planning_urls,
        'content_fn':'profile-content'
    },
    'school_of_info': {
        'url':['https://www.ischool.berkeley.edu/people?name=&role=122'] + \
        ['https://www.ischool.berkeley.edu/people?name=&role=122&degr=All&year[value]&spec=All&emp=&page={}'.format(i)
            for i in range(1, 4)],
        'base':'https://www.ischool.berkeley.edu',
        'url_fn':get_school_of_info_urls,
        'content_fn':'main-content',
        'use_content_id':True
    },
    'me': {
        'url':'http://www.me.berkeley.edu/people/faculty',
        'base':'http://www.me.berkeley.edu',
        'url_fn':get_me_urls,
        'content_fn':'content',
        'use_content_id':True
    },
    'afraamer': {
        'url':'http://africam.berkeley.edu/personnel/faculty',
        'base':'http://africam.berkeley.edu',
        'url_fn':get_afraamer_urls,
        'content_fn':'content clearfix'
    },
    'complit': {
        'url':'http://complit.berkeley.edu/people-category/faculty/',
        'url_fn':get_complit_urls,
        'content_fn':'person-content-wrapper'
    },
    'english': {
        'url':'http://english.berkeley.edu/the_department/faculty',
        'base':'http://english.berkeley.edu',
        'url_fn':get_english_urls,
        'content_fn':'box'
    },
    'geography': {
        'url':'http://geography.berkeley.edu/people/regular-faculty/',
        'url_fn':get_geography_urls,
        'content_fn':'main',
        'use_content_id':True
    },
    'rhetoric': {
        'url':'http://rhetoric.berkeley.edu/people/faculty',
        'base':'http://rhetoric.berkeley.edu',
        'url_fn':get_rhetoric_urls,
        'content_fn':'body-main'
    },
    'philosophy': {
        'url':'https://philosophy.berkeley.edu/people/faculty',
        'base':'https://philosophy.berkeley.edu',
        'url_fn':get_philosophy_urls,
        'content_fn':'row'
    },
}

In [97]:
# test_urls = ['afraamer', 'complit', 'english', 'geography', 'rhetoric', 'philosophy']
test_urls = departments_d.keys()

In [98]:
def get_urls(department):
    return departments_d[department]['urls']

In [99]:
for department in test_urls:
    d = departments_d[department]
    if 'base' in d:
        urls = scrape_urls(d['url'], d['url_fn'], d['base'])
    else:
        urls = scrape_urls(d['url'], d['url_fn'])
    d['urls'] = urls

In [100]:
text_d = {}
flat_urls_d = {}
for department_name in test_urls:
    print('Starting: {}'.format(department_name))
    urls_d = get_urls(department_name)
    for faculty in urls_d:
        url = urls_d[faculty]
        flat_urls_d[faculty] = url
        try:
            text_d[faculty] = scrape_faculty_page(url, department_name)
        except Exception as e:
            print(department_name)
            print(e)

Starting: linguistics
Starting: math
Starting: chem
Starting: plasma_physics
Starting: afraamer
Starting: ib
Starting: philosophy
Starting: mcb
Starting: complit
Starting: eps
Starting: nuclear_physics
Starting: sociology
Starting: biophysics
Starting: cheme
Starting: nuc_eng
Starting: env
Starting: condensed_matter_physics
Starting: are
Starting: english
Starting: gssp
Starting: ced
Starting: geography
Starting: social_welfare
Starting: public_health
Starting: poli_sci
Starting: astro
Starting: anthro
Starting: molecular_physics
Starting: me
Starting: stats
Starting: ieor
Starting: nutri_sci
Starting: astro_physics
astro_physics
Exceeded 30 redirects.
Starting: haas
Starting: rhetoric
Starting: particle_physics
Starting: plant_micro_bio
Starting: civil_env_e
Starting: history
Starting: psych
Starting: econ
Starting: school_of_info
Starting: bioe
Starting: mse
Starting: eecs


In [115]:
rows = []
for faculty in text_d:
    rows.append([faculty, flat_urls_d[faculty], text_d[faculty]])
profile_df = pd.DataFrame(rows, columns=['faculty', 'personal_webpage_url', 'text'])
profile_df.to_csv('../scraped_data/faculty_website_profiles.csv', sep='~', index=False)

In [114]:
text_d

{'John Ellwood': 'John Ellwood Professor Emeritus Research Interests: Budget/Finance Financial Management Public Sector Budgeting Other interests: Member - Policy Council of the Association for Public Policy Analysis and Management Committee member - American Political Science Association Consultant - U.S. government and state government Research Director - Public Policy Institute of California, 1995-97 Profile Updated: January 14, 2015',
 'Iris Mauss': 'Associate Professor Email Address: imauss@berkeley.edu Education: Ph.D., Stanford University Research Area: Social-Personality Laboratory: Emotion & Emotion Regulation Lab at Berkeley Research Interests: Emotion and emotion regulation; psychological health; psychophysiology; sociocultural norms and beliefs; aging and emotion regulation whatshot Research Description place Selected Publications filter_drama Teaching',
 'Polina Lishko': "Polina Lishko Contact Info Assistant Professor of Cell and Developmental Biology Lab Homepage: http://

In [120]:
len(profile_df)

2353

In [9]:
profile_df = pd.read_csv('../scraped_data/faculty_website_profiles.csv', sep='~')

In [11]:
profile_df = pd.read_csv('../scraped_data/faculty_website_profiles.csv', sep='~')
profile_df.head()

,faculty,personal_webpage_url,text
0,John Ellwood,http://sph.berkeley.edu/john-ellwood,John Ellwood Professor Emeritus Research Inter...
1,Iris Mauss,http://psychology.berkeley.edu//people/iris-mauss,Associate Professor Email Address: imauss@berk...
2,Polina Lishko,https://mcb.berkeley.edu/faculty/CDB/lishkop.html,Polina Lishko Contact Info Assistant Professor...
3,Michael Mahoney,http://statistics.berkeley.edu/people/michael-...,Associate Adjunct Professor Email: mmahoney [a...
4,Brent Copen,https://gspp.berkeley.edu/directories/faculty/...,Areas of Expertise Biography


In [12]:
names = set()
for names_l in profile_df['faculty'].apply(lambda s:s.lower().split(' ')).values:
    for name in names_l:
        names.add(name)
stopwords = set(nltk.corpus.stopwords.words('english') + list(string.punctuation))
common_dirty_words = set([
    'center-col',
    'id',
    'content',
    'img',
    'section',
    'page',
    'info', 'home',
    'top', 'directory', 'icon', 'envelope', 'click', 'reach', 'share', 'button',
    'navigation', 'www', 'com', 'html', 'link', 'http', 'edu', 'berkeley', 'website', 'webpage', 'https',
    'false', 'name',
    'email', 'phone', 'room', 'download', 'pdf', 'california', 'welcome',
    'university',  # remove this word or not?
    'printer-friendly', 'linkedin', 'profile', 'contact', 'address', 'biography',
    'alphabetical',  # maybe bad for linguistics
    'publications', 'selected', 'whatshot', 'place',
    'associate', 'assistant', 'researcher', 'research', 'area', 'professor', 'adjunct', 'lab', 'labs',
    'director', 'senior', 'summa', 'cum', 'laude', 'students', 'class', 'staff', 'emeritus', 'degree',
    'lecturer', 'executive', 'leadership', 'visiting', 'ext', 'status', 'duty', 'distinguished',
    'board', 'advisory', 'directors', 'graduating', 'majors', 'affiliate', 'member',
    'also', 'figure', 'group', 'study', 'interest', 'interests', 'interested', 'areas', 'expertise', 'pursuing',
                                                                                                     'name',
    'office', 'hours', 'tuesday', 'monday', 'wednesday', 'thursday', 'friday',
    'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october',
    'november', 'december',
    'etcheverry', 'dwinelle', 'tan', 'latimer',
    'might', 'please', 'faculty', 'like',
    'duke', 'colombia', 'oxford', 'diego', 'yale', 'stanford', 'harvard', 'polytechnic', 'rensselaer',
    'kennedy', 'boston', 'michigan', 'purdue', 'colgate', 'northwestern',
    'curriculue', 'vitae', 'undergraduate', 'graduate',
    'found', 'working', 'held',
    'thought', 'main', 'e-mail', 'personal', 'appointment', 'press', 'cambridge',
    'became', 'prof', 'dot', 'provide', 'coming', 'soon', 'holds', 'hold', 'work', 'works',
    'links', 'kth', 'phd', 'include', 'homepage', 'curriculum', 'format', 'acrobat', 'reader', 'required',
    'editor', 'listing', 'department', 'born', 'cory',
    'courses', 'semester', 'course', 'title', 'syllabus', 'spring', 'fall', 'college',
    'academic', 'publishers', 'springer', 'authored', 'co-authored', 'review', 'papers', 'paper', 'published',
    'editorial', 'committee', 'selection', 'editors', 'editor', 'vol', 'journal',
    'dept', 'currently', 'using', 'focus', 'focuses', 'focused',
    'studying', 'ask', 'behind', 'going', 'still', 'really', 'another',
    'campus', 'online', 'location', 'vcresearch', 'major', 'universities', 'provided', 'operated',
    'tan', 'fax', 'recent', 'novel', 'current', 'search', 'based', 'method', 'methods',
    'technique',
    'various', 'used', 'use',
    'awards', 'prize', 'scholarship', 'scholarships', 'winner', 'recipient', 'honor', 'honors',
    'new', 'changed', 'get', 'example',
    'began', 'following', 'first', 'year', 'second', 'starting', 'served', 'award', 'introductory',
    'received', 'attended', 'joined', 'could', 'enable', 'many', 'involved', 'question', 'questions',
    'problem', 'problems', 'updates', 'center', 'give', 'written', 'extensively', 'along',
    'unique', 'gives', 'different', 'believe', 'shown', 'across', 'able', 'show', 'much', 'available',
    'towards', 'provides', 'loading', 'alloed', 'propose', 'proposed', 'similar', 'instance',
    'multiple', 'primary', 'secondary', 'identify', 'well', 'visit', 'complete', 'list', 'providing',
    'either', 'including', 'effects', 'effect', 'years', 'context', 'site', 'highly', 'would', 'better', 'ways',
    'via', 'xml', 'richards', 'opportunities', 'become', 'epub', 'underlying', 'given', 'span',
    'contribution', 'often', 'uses', 'importance', 'responsible', 'insights', "'kdb", 'edit',

])

grants_common_words = set([
    'continue', 'funding', 'opportunity', 'announcement', 'purpose', 'foa', 'advancing', 'particularly', 'defining', 'observed', 'understanding', 'leading', 'priority', 'high', 'medium', 'achieve', 'program', 'seeks', 'submit', 'programs', 'knowledge', 'substantive', 'collaboration', 'scholars', 'fellowship', 'fellow', 'applicants', 'recognized', 'considerable', 'detailed', 'proposal', 'evaluated', 'applicant', 'qualified', 'members', 'encouraged', 'groups', 'apply', 'applied', 'grants', 'grant', 'awarded', 'award', 'organizations', 'proposing', 'engage', 'position', 'members', 'member', 'mission', 'missions', 'statement', 'goal', 'anticipated', 'bureau', 'necessary', 'notice', 'participants', 'participant', 'acquired', 'individuals', 'populations', 'consequences', 'population', 'student', 'earn', 'two', 'one', 'three', 'individual', 'eligible', 'approved', 'must', 'topic', 'significant', 'least', 'most', 'fellows', 'aims', 'methodology', 'applicability',
])


common_dirty_words = common_dirty_words | names | grants_common_words

In [13]:
def clean_text(text):
    try:
        text = text.lower()
        pattern = '[\r\t\n]'
        repl = ''
        text = re.sub(pattern, repl, text)

        pattern = '[=/\.~_\"“…*]'
        repl = ' '
        text = re.sub(pattern, repl, text)
        
        tokens = nltk.word_tokenize(text)

        l = []
        for token in tokens:
            if token in stopwords or token in common_dirty_words:
                continue
            if len(token) <= 2:
                continue
            contains_num = False
            for i in range(0, 10):
                if str(i) in token:
                    contains_num = True
                    break
            if contains_num:
                continue

            l.append(token)
        return l
    except Exception as e:
#         print(e)
        return []
    
cleaned_text = profile_df['text'].apply(clean_text)
profile_df['cleaned_text'] = cleaned_text

In [14]:
first_last_names = profile_df['faculty'].apply(split_first_last_name)
first_names = [t[0] for t in first_last_names]
last_names = [t[1] for t in first_last_names]
profile_df['first_name'] = first_names
profile_df['last_name'] = last_names
profile_df['faculty'] = profile_df['first_name'] + ' ' + profile_df['last_name']

In [15]:
profile_df['cleaned_str_text'] = profile_df['cleaned_text'].apply(lambda l:' '.join(l))
cleaned_df = profile_df[['faculty', 'personal_webpage_url', 'cleaned_str_text']]

In [16]:
cleaned_df[['faculty', 'personal_webpage_url']].to_csv('../scraped_data/faculty_webpages_names_urls.csv', index=False)

In [17]:
cleaned_df.to_csv('../scraped_data/faculty_website_cleaned_profiles.csv', index=False)

## Still missing some faculty memebers for vcr df so scrape from their personal webpage based on url from vcr

In [18]:
df = pd.read_csv('../scraped_data/faculty_website_cleaned_profiles.csv')

In [19]:
df.head()

,faculty,personal_webpage_url,cleaned_str_text
0,john ellwood,http://sph.berkeley.edu/john-ellwood,budget finance financial management public sec...
1,iris mauss,http://psychology.berkeley.edu//people/iris-mauss,imauss education social-personality laboratory...
2,polina lishko,https://mcb.berkeley.edu/faculty/CDB/lishkop.html,cell developmental biology mcb information cel...
3,michael mahoney,http://statistics.berkeley.edu/people/michael-...,mmahoney stat stat mmahoney
4,brent copen,https://gspp.berkeley.edu/directories/faculty/...,NaN


In [20]:
first_last_names = df['faculty'].apply(split_first_last_name)
first_names = [t[0] for t in first_last_names]
last_names = [t[1] for t in first_last_names]
df['full_name'] = [first_name + ' ' + last_name for first_name, last_name in zip(first_names, last_names)]

In [21]:
len(df)

2353

In [22]:
faculty_vcr_missing = set(df_faculty['full_name']) - set(df['full_name'])
len(faculty_vcr_missing)

646

In [23]:
df.head()

,faculty,personal_webpage_url,cleaned_str_text,full_name
0,john ellwood,http://sph.berkeley.edu/john-ellwood,budget finance financial management public sec...,john ellwood
1,iris mauss,http://psychology.berkeley.edu//people/iris-mauss,imauss education social-personality laboratory...,iris mauss
2,polina lishko,https://mcb.berkeley.edu/faculty/CDB/lishkop.html,cell developmental biology mcb information cel...,polina lishko
3,michael mahoney,http://statistics.berkeley.edu/people/michael-...,mmahoney stat stat mmahoney,michael mahoney
4,brent copen,https://gspp.berkeley.edu/directories/faculty/...,NaN,brent copen


In [24]:
missing_rows = []

name_url_iter = zip(df_faculty['full_name'].tolist(), df_faculty['faculty_site_url'].tolist())

for full_name, url in name_url_iter:
    if full_name in faculty_vcr_missing:
        print(full_name)
        missing_rows.append([full_name, url, scrape_faculty_page_general(url), full_name])

dor abrahamson
richard abrams
kathryn abrams
anthony adams
shahwali ahmadi
asad ahmed
george akerlof
catherine albiston
ronelle alexander
richard allen
barbara allen-diaz
joel altman
nancy klein
margaret anderson
james anderson
william anderson
david anthoff
tomas aragon
krste asanovic
albert russell
robert ashmore
arthur askins
abolhassan astaneh
david attwood
bertrand augst
guitty azarpay
milton azevedo
paola bacchetta
ian bailey
ann banfield
martin banks
patricia baquedano-lópez
robert barde
pranab bardhan
mario barrera
reginald barrett
paul bartlett
robert bartlett
james bartolome
babette barton
gregory barton
gibor basri
hatem bazian
robert bea
alex becker
robert bellah
patricia berger
mark berger
lowell bergman
emilie bergmann
jill berrick
robert berring
vitelmo bertero
joan bieder
thomas biolsi
james bishop
john bishop
leonard bjeldanes
harvey blanch
leo blitz
déborah blocker
joshua bloom
mark blum
ras bodik
victoria bonnell
stephen booth
steven botterill
thomas boyce
jonathan b

In [25]:
valid_faculty = set(df_faculty['full_name'])

In [26]:
missing_df = pd.DataFrame(missing_rows, columns=['faculty', 'personal_webpage_url', 'cleaned_str_text', 'full_name'])
missing_df['cleaned_str_text'] = missing_df['cleaned_str_text'].apply(lambda s:' '.join(clean_text(s)))
df = df.append(missing_df)
# df = df[df['full_name'].apply(lambda name:name in valid_faculty)]

In [29]:
len(df)

3000

In [30]:
df_no_dups = df.drop_duplicates('full_name')

In [32]:
len(df_no_dups)

2923

In [31]:
df_no_dups[['full_name', 'personal_webpage_url', 'cleaned_str_text']]\
    .to_csv('../scraped_data/complete_cleaned_faculty_webpages.csv', index=False)

## Use to check that there are no more faculty members that do not have a text associated with them

In [215]:
merged_df = df_faculty.merge(df, left_on='full_name', 
                             right_on='full_name', sort=True)

In [216]:
faculty_vcr_missing = set(df_faculty['full_name']) - set(merged_df['full_name'])
print(len(faculty_vcr_missing))
sorted(faculty_vcr_missing)

0


[]

In [190]:
faculty_webpages_missing = set(df['full_name']) - set(merged_df['full_name'])
print(len(faculty_webpages_missing))
sorted(faculty_webpages_missing)

1097


['(john) ferrari',
 '(margaret) dhaemers',
 'a.j. gregor',
 'aaron bodoh-creed',
 'aaron mcdaniel',
 'aaron streets',
 'abby scott',
 'abhay parekh',
 'abhishek nagaraj',
 'abolhassan astaneh-asl',
 'adair morse',
 'adam berman',
 'adam leipzig',
 'adam lucas',
 'adam wolisz',
 'adityanand guntuboyina',
 'ajay bam',
 'ajay manthripragada',
 'alain janvry',
 'alan code',
 'alan hammond',
 'alan hubbard',
 'alan ross',
 'alastair iles',
 'alberto grünbaum',
 'alberto todeschini',
 'alessandro chiesa',
 "alex d'amour",
 'alex hughes',
 'alex marrs',
 'alex pines',
 'alexander gelber',
 'alexander givental',
 'alexander nezlobin',
 'alexander paulin',
 'alexander skabardonis',
 'alexandra meier',
 'alexandra minnis',
 'alexandru scorpan',
 'alexei (alyosha)',
 'alexis guerra',
 'ali ghodsi',
 'allan jacobs',
 'allan marks',
 'allan sindler',
 'alva noë',
 'amin azzam',
 'amir kermani',
 'amit bhattacharyya',
 'amy garlin',
 'amy herr',
 'amy kyle',
 'amy slater',
 'anand chokkalingam',
 'a

In [169]:
merged_df

,faculty_name,faculty_profile_url,l_expertise,department,title_name,faculty_site_url,lab_url,faculty_email,description,description_links,...,description_teaser_5,first_name_x,last_name_x,full_name,lowercase_faculty_name_x,faculty,cleaned_str_text,first_name_y,last_name_y,lowercase_faculty_name_y
0,Aaron S. Edlin,/faculty/aaron-s-edlin,"economics,industrial organization,regulation,a...",Department of Economics,Professor of Economics and Professor of Law,http://works.bepress.com/aaron_edlin/,NaN,edlin@law.berkeley.edu,Aaron Edlin is a leading expert in economics ...,NaN,...,NaN,aaron,edlin,aaron edlin,aaron s. edlin,Aaron Edlin,curriculum vitae homepage aaron edlin richard ...,aaron,edlin,aaron edlin
1,Aaron Fisher,/faculty/aaron-fisher,"anxiety,depression,personalized medicine,psych...",Department of Psychology,Assistant Professor,http://psychology.berkeley.edu/people/aaron-fi...,http://www.dynamicpsychlab.com/,afisher@berkeley.edu,The broad goal of my research is to understan...,NaN,...,NaN,aaron,fisher,aaron fisher,aaron fisher,Aaron Fisher,assistant professor address afisher office tol...,aaron,fisher,aaron fisher
2,Abby Dernburg,/faculty/abby-dernburg,"genomics,chromosome remodeling and reorganizat...",Department of Molecular & Cell Biology,Professor in Residence of Cell and Development...,http://mcb.berkeley.edu/faculty/CDB/dernburga....,NaN,afdernburg@lbl.gov,For genetic information to be transmitted fro...,NaN,...,NaN,abby,dernburg,abby dernburg,abby dernburg,Abby Dernburg,abby dernburg info howard hughes medical insti...,abby,dernburg,abby dernburg
3,Abdul JanMohamed,/faculty/abdul-janmohamed,critical theory; theory of subjection; postcol...,Department of English,Professor of English,http://english.berkeley.edu/profiles/41,NaN,abduljm@berkeley.edu,Born and raised in Kenya and educated in the ...,NaN,...,NaN,abdul,janmohamed,abdul janmohamed,abdul janmohamed,Abdul R. JanMohamed,abdul janmohamed professor hearst annex tues t...,abdul,janmohamed,abdul r. janmohamed
4,Abhishek Kaicker,/faculty/abhishek-kaicker,"south asia,mughal,early modern,cities,history,...",Department of History,Assistant Professor,http://history.berkeley.edu/people/abhishek-ka...,NaN,kaicker@berkeley.edu,Abhishek Kaicker is a historian of South Asia...,NaN,...,NaN,abhishek,kaicker,abhishek kaicker,abhishek kaicker,Abhishek Kaicker,real name abhishek kaicker assistant professor...,abhishek,kaicker,abhishek kaicker
5,Adam Arkin,/faculty/adam-arkin,"systems and synthetic biology,environmental mi...",Department of Bioengineering,Professor of Bioengineering,http://bioeng.berkeley.edu/people/cv?facultyid...,http://genomics.lbl.gov,aparkin@berkeley.edu,The Arkin laboratory for systems and syntheti...,NaN,...,NaN,adam,arkin,adam arkin,adam arkin,Adam Arkin,dean richard newton memorial professor bioengi...,adam,arkin,adam arkin
6,Adib Kanafani,/faculty/adib-kanafani,"transportation economics,transportation planni...",Department of Civil and Environmental Engineering,Professor of the Graduate School & Cahill Prof...,http://www.ce.berkeley.edu/faculty/faculty.php...,NaN,kanafani@berkeley.edu,NaN,NaN,...,NaN,adib,kanafani,adib kanafani,adib kanafani,Adib Kanafani,professor graduate school kanafani fax mclaugh...,adib,kanafani,adib kanafani
7,Adina M. Merenlender,/faculty/adina-m-merenlender,"conservation biology,water resources,land use ...","Environmental Science, Policy & Management","Professor of Environmental Science, Policy, an...",http://ourenvironment.berkeley.edu/people_prof...,http://ucanr.org/sites/merenlender/,adinam@berkeley.edu,My research focuses on quantifying the relati...,http://ucanr.org/sites/UCCNP/,...,NaN,adina,merenlender,adina merenlender,adina m. merenlender,Adina Merenlender,NaN,adina,merenlender,adina merenlender
8,Adrian Aguilera,/faculty/adrian-aguilera,"culture and ses and mental health,mental healt...",School of Social Welfare,Assistant Professor,http://socialwelfare.berkeley.edu/adrian-aguilera,NaN,aguila@berkeley.edu,My research foc